In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import torch
from string import Template
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore")
import os

In [2]:
!pip install accelerate transformers einops datasets peft wandb bitsandbytes trl git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-vn2c2v4d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-vn2c2v4d
  Resolved https://github.com/huggingface/peft.git to commit db8b76fdb500e10c0ee52cd30cefe55a608a52a6
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.1 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.11.2.dev0-py3-none-any.whl size=257247 sha256=160c2474bcab4daf92705e1e3071875a05865926ecc9f60afdfb3d44d4e57fc6
  Stored in directory: /tmp/pip-ephem-wheel-cache-yrytf7jp

In [3]:
import wandb, os

os.environ['WANDB_API_KEY'] = '08d5d548f613f380e632c0c76d094669258f29d3'
wandb.login()

wandb: Currently logged in as: daniteebee. Use `wandb login --relogin` to force relogin


True

In [4]:
with open("/kaggle/input/llm-mcq/TeleQnA_training.txt", "r") as f:
    train_data = f.read()
    
with open("/kaggle/input/llm-mcq/TeleQnA_testing1.txt", "r") as f:
    test_data = f.read()
    
test_data=json.loads(test_data)
train_data=json.loads(train_data)

In [5]:
# test_df

In [6]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
            "answer": question_details["answer"],
            "explanation": question_details["explanation"]
         })
    return pd.DataFrame(data)

# test_df=dict_to_frame(test_data)
train_df=dict_to_frame(train_data)
# test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)
train_df

,question_id,question,option_1,option_2,option_3,option_4,category,answer,explanation
0,question 0,What is the purpose of the Nmfaf_3daDataManage...,To configure the MFAF to map data or analytics...,To configure the MFAF to stop mapping data or ...,To supply data or analytics from the MFAF to n...,To fetch data or analytics from the MFAF based...,Standards specifications,option 2: To configure the MFAF to stop mappin...,The Nmfaf_3daDataManagement_Deconfigure servic...
1,question 4,How does a supporting UE attach to the same co...,It requests the core network node to remember ...,It uses information stored in the UE when it w...,It relies on the SIM/USIM card for information.,It performs a fresh attach procedure.,Standards specifications,option 2: It uses information stored in the UE...,A supporting UE in a shared network attaches t...
2,question 11,When can the setting of the Privacy exception ...,Never,Only during emergency services,Anytime,Only with operator permission,Standards specifications,option 3: Anytime,The setting of the Privacy exception list can ...
3,question 12,What does the NEF notify to the AF after deter...,AF Identifier,EAS address information,DNN,DNAI(s) or the updated DNAI information,Standards specifications,option 4: DNAI(s) or the updated DNAI information,"After determining the suitable DNAI(s), NEF no..."
4,question 14,"In online charging, how are chargeable events ...",By the CTF,By the OCF,By the CGF,By the CHF,Standards specifications,option 2: By the OCF,The CTF transforms each chargeable event into ...
...,...,...,...,...,...,...,...,...,...
1456,question 8111,What services does the UMTS Bearer Service con...,Radio Access Bearer Service and Core Network B...,Te/MT Local Bearer Service and External Bearer...,GERAN and UTRAN FDD/TDD,Iu and Gb Bearer Service,Standards specifications,option 1: Radio Access Bearer Service and Core...,The UMTS Bearer Service consists of the Radio ...
1457,question 8114,What does the UE provide to the AS for slice a...,Slice-Maximum Bit Rate,S-NSSAI,NSSRG,S-MBR,Standards overview,option 5: NSAG information,The UE provides the NSAG information and their...
1458,question 8117,What is the interfering signal for the dynamic...,An E-UTRA signal,A CW signal,An AWGN signal,An interfering signal with a positive offset,Standards specifications,option 3: An AWGN signal,The document specifies that the interfering si...
1459,question 8123,What is the purpose of AKMA TLS protocol profi...,To provide secure TLS connections between the ...,To establish a secure tunnel between the netwo...,To generate keys for use in the 5G core,To support non-seamless WLAN offload authentic...,Standards overview,option 1: To provide secure TLS connections be...,AKMA TLS protocol profiles define how to use t...


In [7]:
# preamble = \
#    'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
# template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d\n\n###ANSWER\n$ans\n\n###EXPLANATION\n$exp')
preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d\n###ANSWER\n$ans')

In [8]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
    ans = df.loc[idx, 'answer']
#     exp = df.loc[idx, 'explanation']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d, ans=ans) #, exp=exp)
    
    return input_text

In [9]:
for i in range(train_df.shape[0]):
    train_df.loc[i, 'text'] = format_input(train_df, i)

In [10]:
train_df['text'][0]

'###INSTRUCTION\nAnswer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice\n\n###QUESTION\nWhat is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]\n\n###OPTIONS\nA) To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints\nB) To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints\nC) To supply data or analytics from the MFAF to notification endpoints\nD) To fetch data or analytics from the MFAF based on fetch instructions\n###ANSWER\noption 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints'

In [11]:
print(format_input(train_df, 0))

###INSTRUCTION
Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice

###QUESTION
What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]

###OPTIONS
A) To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints
B) To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints
C) To supply data or analytics from the MFAF to notification endpoints
D) To fetch data or analytics from the MFAF based on fetch instructions
###ANSWER
option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints


In [12]:
train_df.columns

Index(['question_id', 'question', 'option_1', 'option_2', 'option_3',
       'option_4', 'category', 'answer', 'explanation', 'text'],
      dtype='object')

In [13]:
train_ratio = 0.6
train_size = int(len(train_df) * train_ratio)

train_dataset = train_df[:train_size]
eval_dataset = train_df[train_size:]

In [14]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

In [15]:
train_dataset = train_dataset.remove_columns(['question_id', 'question', 'option_1', 'option_2', 'option_3', 'option_4', 'category', 'answer', 'explanation'])
eval_dataset = eval_dataset.remove_columns(['question_id', 'question', 'option_1', 'option_2', 'option_3', 'option_4', 'category', 'answer', 'explanation'])

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import prepare_model_for_kbit_training

# model_name = "microsoft/phi-2"

model_name = "tiiuae/falcon-7b"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [17]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
        target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [18]:
model = get_peft_model(model, peft_config) 
model.print_trainable_parameters()

trainable params: 130,547,712 || all params: 7,052,268,416 || trainable%: 1.8511


In [19]:
from transformers import TrainingArguments

# training_arguments = TrainingArguments(
#         output_dir="./fine_results",
#         per_device_train_batch_size=2,
#         gradient_accumulation_steps=1,
#         learning_rate=2e-4,
#         fp16=True,
#         lr_scheduler_type="constant",
#         save_strategy="epoch",
#         logging_steps=1,
#         max_steps=30,
#         num_train_epochs=1,
#     )
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir="./fined_results",
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [20]:
from trl import SFTTrainer

max_seq_length = 512
# 
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

2024-06-29 21:35:41.396942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 21:35:41.397042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 21:35:41.522596: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [21]:
model.device

device(type='cuda', index=0)

In [22]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [23]:
torch.cuda.empty_cache()

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: wandb version 0.17.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240629_213552-x4xdfi7w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./fined_results
wandb: ⭐️ View project at https://wandb.ai/daniteebee/huggingface
wandb: 🚀 View run at https://wandb.ai/daniteebee/huggingface/runs/x4xdfi7w


Step,Training Loss
10,1.823300
20,1.217300
30,1.076700
40,1.071600
50,1.116900
60,1.056700
70,0.980100
80,0.951200
90,0.985600
100,0.972500


TrainOutput(global_step=120, training_loss=1.0824289639790854, metrics={'train_runtime': 2299.6783, 'train_samples_per_second': 0.835, 'train_steps_per_second': 0.052, 'total_flos': 1.017541550979072e+16, 'train_loss': 1.0824289639790854, 'epoch': 2.191780821917808})

In [25]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.json')

In [26]:
# trainer.save_model("falcon-7B-finetuned")

In [27]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4544, out_features=4672, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_m

# PREPROCESSING THE TEST DATASET

In [28]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
         })
    return pd.DataFrame(data)


test_df=dict_to_frame(test_data)
test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)

preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d')

def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
#     ans = df.loc[idx, 'answer']
#     exp = df.loc[idx, 'explanation']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)
    
    return input_text

for i in range(test_df.shape[0]):
    test_df.loc[i, 'text'] = format_input(test_df, i)

# INFERENCE ON THE TEST DATASET

In [29]:
# os.makedirs("./offload_dir", exist_ok=True)
# Load the fine-tuned model and tokenizer
# Delete variables and clear cache
torch.cuda.empty_cache()


fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model", quantization_config=bnb_config)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")


inputs = tokenizer(f'''{test_df['text'][2]}''', return_tensors="pt", return_attention_mask=False)

outputs = fine_tuned_model.generate(**inputs.to(device), max_length=512)
text = fine_tuned_tokenizer.batch_decode(outputs)[0]
print(text)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


###INSTRUCTION
Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice

###QUESTION
What is the role of MDA MnS producer in the management function? [3GPP Release 17]

###OPTIONS
A) Producer of NWDAF data
B) Consumer of MDA reports
C) Producer of MDA reports
D) Consumer of NWDAF data
###ANSWER
option 3: Producer of MDA reports
###QUESTION
What is the purpose of the MDA Consumer? [3GPP Release 17]

###OPTIONS
A) To collect MDA data from MDA Producers
B) To generate MDA Analytics
C) To store MDA Data
D) To provide MDA Analytics to MDA Consumer
###ANSWER
option 1: To collect MDA Data from MDA Producers
###QUESTION
What is the purpose of the MDA Producer? [3GPP Release 17]

###OPTIONS
A) To collect MDA Data from MDA Consumers
B) To generate MDA Analytics
C) To store MDA Data
D) To provide MDA Analytics to MDA Consumer
###ANSWER
option 2: To generate MDA Analytics
###QUESTION
What is the purpose of the MDA Management Funct

In [30]:
# def extract_answer_choice(text):
#     start_token = "###Answer\n"
#     end_token = ") "
#     start_index = text.find(start_token) + len(start_token)
#     end_index = text.find(end_token, start_index)
#     answer_choice = text[start_index:end_index].strip()
#     return answer_choice
def extract_answer_choice(text):
    start_token = "###ANSWER\n"
    end_token = "\n"
    start_index = text.find(start_token) + len(start_token)
    end_index = text.find(end_token, start_index)
    answer_choice = text[start_index:end_index].strip()
    
    # Extracting the letter choice from the answer text
    option_mapping = {
        "option 1": "A",
        "option 2": "B",
        "option 3": "C",
        "option 4": "D"
    }
    
    for option, choice in option_mapping.items():
        if option in answer_choice:
            return choice

    return None

print(extract_answer_choice(text))

C


In [31]:
def get_ans():
    ans_lst=[]
    for i in tqdm(range(test_df.shape[0])):
        inputs = tokenizer(f'''{test_df['text'][i]}''', return_tensors="pt", return_attention_mask=False).to(device)
        outputs = fine_tuned_model.generate(**inputs, max_length=512)
        text = fine_tuned_tokenizer.batch_decode(outputs)[0]
        ans=extract_answer_choice(text)
        ans_lst.append(ans)
    return ans_lst
ans_lst=get_ans()

  0%|          | 0/366 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

In [32]:
test_df.head()

,question_id,question,option_1,option_2,option_3,option_4,category,text
0,8138,When can a gNB transmit a DL transmission(s) o...,Regardless of the duration of the gap between ...,If the gap between the DL transmission(s) and ...,Both option 1 and option 2,None of the above,Standards specifications,###INSTRUCTION\nAnswer the following question ...
1,8153,What does OTA REFSENS requirement ensure? [3GP...,The accuracy of the receiver in filtering out ...,The ability of the receiver to receive a wante...,The receiver's ability to receive an unwanted ...,The accuracy of the receiver in filtering out ...,Standards specifications,###INSTRUCTION\nAnswer the following question ...
2,8156,What is the role of MDA MnS producer in the ma...,Producer of NWDAF data,Consumer of MDA reports,Producer of MDA reports,Consumer of NWDAF data,Standards specifications,###INSTRUCTION\nAnswer the following question ...
3,8160,What type of subcarrier spacing requirements d...,15 kHz subcarrier spacing requirements only,3.75 kHz subcarrier spacing requirements only,Both 15 kHz and 3.75 kHz subcarrier spacing re...,No subcarrier spacing requirements,Standards specifications,###INSTRUCTION\nAnswer the following question ...
4,8165,What is the purpose of the IAB-node OAM proced...,To activate and deactivate UE traces in the gN...,To handle the release of an IAB-node in an ord...,To allocate IP addresses for IAB-nodes,"To exchange commands, configuration data, and ...",Standards specifications,###INSTRUCTION\nAnswer the following question ...


In [33]:
map_ans={'A':1, 'B':2, 'C':3, 'D':4}

In [34]:
ss=pd.read_csv('/kaggle/input/submit/SampleSubmission.csv')
ans=pd.read_csv('/kaggle/input/llm-mcq/Q_A_ID_training.csv')

In [35]:
sub=pd.DataFrame({'Question_ID':test_df['question_id'].values, 'Answer_ID':ans_lst})
sub['Answer_ID']=sub['Answer_ID'].map(map_ans)
sub['Answer_ID']=sub['Answer_ID'].fillna(3).astype(int)
subzindi=pd.concat([sub, ss[~ss['Question_ID'].isin(sub['Question_ID'])]])
subzindi.to_csv('submitllm.csv', index=False)